<a href="https://colab.research.google.com/github/Kimjoozero/deep_learning/blob/main/HW2_60192180_%EA%B9%80%EC%A3%BC%EC%98%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [42]:
!pwd

/content


In [43]:
# lms에서 sample_64_half.npz 파일을 다운로드 하신 후에 구글 drive에 업로드 하세요.
# 업로드 위치는 최상위 디렉토리인 My Drive 입니다.

In [44]:
from google.colab import drive # google drive와 연동하기 위한 라이브러리
drive.mount('/content/drive') # google drive와 연결(mount) 완료

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [45]:
import numpy as np
filename = '/content/drive/My Drive/sample_64_half.npz'
np_files = np.load(filename)

# 학습용 데이터셋
train_data = np_files['x_train']
train_labels = np_files['y_train']

# 검증용 데이터셋
valid_data = np_files['x_valid']
valid_labels = np_files['y_valid']

# 학습 데이터의 형태
train_data.shape

(8000, 4096, 1)

In [46]:
# 데이터는 4096개의 숫자를 가지는 데이터이다.
train_data[0]

array([[ 77],
       [ 90],
       [144],
       ...,
       [  0],
       [  0],
       [  0]], dtype=uint8)

In [47]:
# 정답 데이터의 형식 : 숫자 0 또는 1
train_labels[0:10]

array([0, 0, 1, 1, 0, 1, 0, 1, 0, 1])

In [53]:
from tensorflow import keras
from tensorflow.keras import layers

model = keras.Sequential(name="binary_classification_model")

model.add(keras.Input(shape=(64, 64, 1), name="input_layer"))
model.add(layers.Conv2D(32, kernel_size=(3, 3), activation="relu"))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu', name='hidden_dense_layer'))

model.add(layers.Dense(1, activation="sigmoid", name="output_layer"))

model.summary()

model.compile(loss="binary_crossentropy",
              optimizer="adam",
              metrics=["accuracy"])
print("\n모델 구성 및 컴파일 완료.")

Model: "binary_classification_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_4 (Conv2D)               │ (None, 62, 62, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 31, 31, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_4 (Flatten)             │ (None, 30752)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ hidden_dense_layer (Dense)      │ (None, 64)             │     1,968,192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output_layer (Dense)            │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,968,577 (7.51 MB)

 Trainable params: 1,968,577 (7.51 MB)

 Non-trainable params: 0 (0.00 B)


모델 구성 및 컴파일 완료.


In [54]:
import numpy as np
from sklearn.model_selection import train_test_split


print("Original data shape:", train_data.shape)


if train_data.ndim == 3 and train_data.shape[1] == 4096:
    num_samples = train_data.shape[0]
    train_data_reshaped = train_data.reshape((num_samples, 64, 64, 1))
elif train_data.ndim == 2 and train_data.shape[1] == 4096:
    num_samples = train_data.shape[0]
    train_data_reshaped = train_data.reshape((num_samples, 64, 64, 1))
elif train_data.ndim == 3 and train_data.shape[-1] == 1:
    train_data_reshaped = train_data
else:
    print("Warning: Data shape is not in expected format (8000, 4096, 1). Attempting reshape to (8000, 64, 64, 1)")
    num_samples = train_data.shape[0]
    try:
        train_data_reshaped = train_data.reshape((num_samples, 64, 64, 1))
    except ValueError:
        print("Error: Could not reshape data to (num_samples, 64, 64, 1). Please check your input data shape.")
        train_data_reshaped = train_data


print("Reshaped data shape:", train_data_reshaped.shape)


train_data_normalized = train_data_reshaped.astype('float32') / 255.0
print("Data normalized.")


try:
    seed_value
except NameError:
    seed_value = 42

x_train, x_val, y_train, y_val = train_test_split(
    train_data_normalized,
    train_labels,
    test_size=0.2,
    random_state=seed_value,
    stratify=train_labels
)
print("Data split into training and validation sets:")
print("x_train shape:", x_train.shape, "| y_train shape:", y_train.shape)
print("x_val shape:", x_val.shape, "| y_val shape:", y_val.shape)

Original data shape: (8000, 4096, 1)
Reshaped data shape: (8000, 64, 64, 1)
Data normalized.
Data split into training and validation sets:
x_train shape: (6400, 64, 64, 1) | y_train shape: (6400,)
x_val shape: (1600, 64, 64, 1) | y_val shape: (1600,)


In [55]:
from tensorflow.keras.callbacks import ModelCheckpoint

epochs = 20
batch_size = 128


checkpoint_filepath = '/content/drive/MyDrive/best_model.keras'
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)
print(f"Starting model training for {epochs} epochs...")
history = model.fit(
    x_train, y_train,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=(x_val, y_val),
    callbacks=[model_checkpoint_callback]
)
print("\nModel training finished.")



Starting model training for 20 epochs...
Epoch 1/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 19s 363ms/step - accuracy: 0.6268 - loss: 0.8530 - val_accuracy: 0.7919 - val_loss: 0.4678
Epoch 2/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 19s 374ms/step - accuracy: 0.8045 - loss: 0.4229 - val_accuracy: 0.8156 - val_loss: 0.3953
Epoch 3/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 22s 401ms/step - accuracy: 0.8621 - loss: 0.3468 - val_accuracy: 0.8294 - val_loss: 0.3603
Epoch 4/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 19s 376ms/step - accuracy: 0.8891 - loss: 0.2964 - val_accuracy: 0.8731 - val_loss: 0.3318
Epoch 5/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 22s 399ms/step - accuracy: 0.9114 - loss: 0.2494 - val_accuracy: 0.8650 - val_loss: 0.3066
Epoch 6/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 19s 362ms/step - accuracy: 0.9191 - loss: 0.2223 - val_accuracy: 0.8875 - val_loss: 0.2849
Epoch 7/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 19s 374ms/step - accuracy: 0.9313 - loss: 0.1937 - val_accuracy: 0.8612 - val_loss: 0.3199
Epoch 8/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 21s 393ms/step - a

In [56]:

final_train_acc = history.history['accuracy'][-1]
final_val_acc = history.history['val_accuracy'][-1]
print("Using results from the final epoch.")


final_score = (final_train_acc + final_val_acc) / 2

print("\n--- Final Results ---")
print(f"# Accuracy : {final_train_acc*100:.2f}%")
print(f"# Validation Accuracy : {final_val_acc*100:.2f}%")
print(f"# Final Score : {final_score*100:.2f}%")

Using results from the final epoch.

--- Final Results ---
# Accuracy : 97.08%
# Validation Accuracy : 89.75%
# Final Score : 93.41%
